In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import requests
plt.rc("figure", figsize=(10, 6))
%matplotlib
%matplotlib inline

Using matplotlib backend: <object object at 0x0000020A63924E70>


In [2]:
df = pd.read_csv("examples/ex1.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'examples/ex1.csv'

In [ ]:
df

In [ ]:
df2 = pd.read_csv("examples/ex5.csv")
df2

In [ ]:
df2 = pd.read_csv("examples/ex5.csv", na_values=["1"])
df2

In [ ]:
with pd.read_csv("examples/ex6.csv", chunksize=5) as reader:
    reader
    for chunk in reader:
        print(chunk)

In [ ]:
url = "https://imdb-api.com/en/API/Top250Movies/k_qvgt5hrx"

In [ ]:
resp = requests.get(url)
resp.raise_for_status()
resp

In [ ]:
data = resp.json()

In [ ]:
topMovies = pd.DataFrame(data["items"])

In [ ]:
top5Movies = topMovies.loc[:4,["id","rank","title","year"]]
top5Movies

In [ ]:
topMovies.to_json("examples/imdb_top250movies.json")

In [ ]:
topMovies

In [ ]:
top5Movies.iloc[1:4,1] = np.nan
top5Movies.iloc[1:2,2] = np.nan
top5Movies

In [ ]:
top5Movies.dropna()

In [ ]:
top5Movies.fillna({"rank":"0", "title": "<Not Available>"})


In [ ]:
top5Movies.isna()

In [ ]:
top5Movies.replace({"The Dark Knight":"Batman2"})

In [ ]:
def mapIndex(x):
        return "Platz: " + str(int(x)+1) 

In [ ]:
top5Movies.rename(index=mapIndex, columns=str.title)

In [ ]:
crew = topMovies["crew"].head()
crew

In [ ]:
crew_split = crew.str.split(pat=",")
crew_split

In [ ]:
top5MoviesIds = top5Movies["id"]
top5MoviesIds

In [ ]:
def getRatingsTable(ids):
    ratings = []
    baseUrl = "https://imdb-api.com/en/API/Ratings/k_qvgt5hrx/"
    for idx, val in ids.items():
        resp2 = requests.get(baseUrl+val)
        ratings.append(resp2.json())
    return ratings

In [ ]:
ratings = getRatingsTable(top5MoviesIds)
ratings

In [ ]:
ratingsDf = pd.DataFrame(ratings).drop(["errorMessage"], axis=1)
ratingsDf

In [ ]:
pd.merge(top5Movies, ratingsDf, left_on="id", right_on="imDbId")

In [ ]:
firstPart = topMovies.iloc[:5]
firstPart

In [ ]:
secondPart = topMovies.iloc[100:105]
secondPart

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2),index=["a", "c"],columns=["three", "four"])

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1,df2], axis="columns", join="inner")

In [ ]:
pd.DataFrame(crew_split.to_dict())

In [ ]:
crew_hierarchical = pd.DataFrame(crew_split.to_dict()).unstack()
crew_hierarchical

In [ ]:
crew_hierarchical[0]

In [ ]:
crew_hierarchical[:,0]

In [ ]:
resp3 = requests.get("https://imdb-api.com/en/API/BoxOfficeAllTime/k_qvgt5hrx")
resp3.raise_for_status()
boxOfficeDf = pd.DataFrame(resp3.json()["items"])
boxOfficeDf.groupby("year")

In [ ]:
boxOfficeDf.groupby("year")["title"].count()

In [ ]:
groupByDictionaryExample = pd.DataFrame(np.random.standard_normal((5, 5))*1000,
        columns=["Deutschland", "Frankreich", "Japan", "China", "Vietnam"],
        index=["Film1", "Film2", "Film3", "Film4", "Film5"])
groupByDictionaryExample

In [ ]:
mapping = {"Deutschland": "Europa", "Frankreich":"Europa", "Japan": "Asien", "China":"Asien","Vietnam":"Asien"}

In [ ]:
groupByDictionaryExample.groupby(mapping, axis=1).sum()

In [ ]:
groupByDictionaryExample.groupby(mapping, axis=1).quantile(q=1)

In [ ]:
def dollarStringToNumber(value):
    return (value.str.replace("[$,.]","", regex = True))

In [ ]:
boxOfficeDf.groupby("year")["worldwideLifetimeGross"].count()

In [ ]:
boxOfficeDf["worldwideLifetimeGross"] = boxOfficeDf["worldwideLifetimeGross"].str.replace("[$,.]","", regex = True).astype(float)

In [ ]:
boxOfficeDf.groupby("year")["worldwideLifetimeGross"].sum()

In [ ]:
boxOfficeDf

In [ ]:
def get_stats(group):
    return pd.Series(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
)

In [ ]:
boxOfficeDf.groupby("year")["worldwideLifetimeGross"].apply(get_stats).unstack()

In [ ]:
data = boxOfficeDf.pivot_table(aggfunc=["count", "mean", "sum"],index=["year"],values=["worldwideLifetimeGross"])

In [ ]:
fig, ax = plt.subplots(3, sharex=True)

In [ ]:
col =  ["Anzahl Filme","Durchschnitteinnahmen pro Film","Gesamteinnahmen weltweit"]
data.columns = col
color = ["blue","orange","green"]
for idx, label in enumerate(col):
    if(idx != 0):
        ax[idx].set_ylabel("In Euro")
    ax[idx].plot(data[label], color = color[idx])
    ax[idx].set_title(label)

ax[-1].set_xticks(ax[-1].get_xticks(), ax[-1].get_xticklabels(), rotation=60, ha='right')
ax[-1].set_xlabel("Jahr")


In [ ]:
fig.subplots_adjust(wspace=0, hspace=0.5)
fig

In [ ]:
data.plot(subplots=True)